<a href="https://colab.research.google.com/github/deenyse/VSB_ZSU/blob/main/Copy_of_UASS_3_07_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import networkx as nx
from statistics import mean
import operator # Import pro seřazení slovníku

# --- Definice funkcí pro analýzu sítě ---

# Vrací seznam vrcholů
def nodes(G: nx.Graph) -> list:
    return list(G.nodes())


# Vrací seznam hran
def edges(G: nx.Graph) -> list:
    return list(G.edges())


# Vypočítá hustotu sítě
def net_density(G: nx.Graph) -> float:
    # Počet hran / počet možných hran v neorientovaném grafu
    num_nodes = len(nodes(G))
    # Vzorec pro neorientovaný graf
    return len(edges(G)) / ((num_nodes * (num_nodes - 1)) / 2) if num_nodes > 1 else 0


# Vrací stupeň daného vrcholu
def local_degree(G: nx.Graph, node) -> int:
    return G.degree[node]


# Vrací seznam stupňů všech vrcholů v síti
def nodes_degrees(G: nx.Graph) -> list[int]:
    return [local_degree(G, i) for i in nodes(G)]


# Vypočítá průměrný stupeň sítě
def average_degree(G: nx.Graph) -> float:
    return mean(nodes_degrees(G))


# Vrací nejmenší stupeň v síti
def min_degree(G: nx.Graph) -> int:
    return min(nodes_degrees(G))


# Vrací největší stupeň v síti
def max_degree(G: nx.Graph) -> int:
    return max(nodes_degrees(G))


# Vypočítá lokální shlukovací koeficient pro daný vrchol
def local_clustering_coefficient(G: nx.Graph, node) -> float:
    k_i = local_degree(G, node) # Stupeň vrcholu
    if k_i > 1:
        # Použijeme vestavěnou a optimalizovanou funkci z NetworkX
        return nx.clustering(G, node)
    else:
        return 0.0


# Vypočítá průměrný shlukovací koeficient pro celou síť
def average_clustering_coefficient(G: nx.Graph) -> float:
    # Spočítáme průměr z lokálních koeficientů všech vrcholů
    values = [local_clustering_coefficient(G, n) for n in nodes(G)]
    return mean(values)


# Vrací Closeness Centrality pro daný vrchol
def nx_closeness(G: nx.Graph, node):
    closeness_dict = nx.closeness_centrality(G)
    return closeness_dict[node]


# Vrací Betweenness Centrality pro daný vrchol
def nx_betweenness(G: nx.Graph, node):
    betweenness_dict = nx.betweenness_centrality(G)
    return betweenness_dict[node]


# --- Hlavní funkce pro provedení a vytisknutí analýzy ---

def analyze_network(file_path: str, network_name: str, delimiter: str = ','):
    print(f"--- ANALÝZA SÍTĚ: {network_name.upper()} ---")

    G_raw: nx.Graph = nx.read_edgelist(file_path, delimiter=delimiter, nodetype=str)

    G = nx.Graph()
    for u, v in G_raw.edges():
        G.add_edge(u.strip(), v.strip())


    print("\n## GLOBÁLNÍ VLASTNOSTI ##")
    print(f"Velikost sítě: {len(nodes(G))} vrcholů a {len(edges(G))} hran.")
    print(f"Hustota sítě: {net_density(G):.4f}")
    print(f"Průměrný stupeň: {average_degree(G):.4f}")
    print(f"Minimální stupeň: {min_degree(G)}")
    print(f"Maximální stupeň: {max_degree(G)}")
    print(f"Průměrný shlukovací koeficient: {average_clustering_coefficient(G):.4f}")

    # 2. Lokální vlastnosti pro 2 nejvýznamnější vrcholy
    # Najdeme vrchol s nejvyšším stupněm a nejvyšší betweenness centralitou
    degrees = dict(G.degree())
    betweenness = nx.betweenness_centrality(G)

    node_max_degree = max(degrees.items(), key=operator.itemgetter(1))[0]
    node_max_betweenness = max(betweenness.items(), key=operator.itemgetter(1))[0]

    chosen_nodes = {node_max_degree, node_max_betweenness}

    print("\n## LOKÁLNÍ VLASTNOSTI (PRO VYBRANÉ VRCHOLY) ##")
    for node in chosen_nodes:
        print(f"\nAnalýza vrcholu: '{node}'")
        print(f"  - Stupeň: {local_degree(G, node)}")
        print(f"  - Shlukovací koeficient: {local_clustering_coefficient(G, node):.4f}")
        print(f"  - Closeness Centrality: {nx_closeness(G, node):.4f}")
        print(f"  - Betweenness Centrality: {nx_betweenness(G, node):.4f}")

    print("\n--- KONEC ANALÝZY ---\n")



analyze_network("sample_data/Dolphins.csv", "Dolphins social network", delimiter=';')
analyze_network("sample_data/Les_Miserables.csv", "Les Miserables character co-occurrence", delimiter=';')


--- ANALÝZA SÍTĚ: DOLPHINS SOCIAL NETWORK ---

## GLOBÁLNÍ VLASTNOSTI ##
Velikost sítě: 62 vrcholů a 159 hran.
Hustota sítě: 0.0841
Průměrný stupeň: 5.1290
Minimální stupeň: 1
Maximální stupeň: 12
Průměrný shlukovací koeficient: 0.2590

## LOKÁLNÍ VLASTNOSTI (PRO VYBRANÉ VRCHOLY) ##

Analýza vrcholu: '36'
  - Stupeň: 7
  - Shlukovací koeficient: 0.0476
  - Closeness Centrality: 0.4178
  - Betweenness Centrality: 0.2482

Analýza vrcholu: '14'
  - Stupeň: 12
  - Shlukovací koeficient: 0.2576
  - Closeness Centrality: 0.3765
  - Betweenness Centrality: 0.0620

--- KONEC ANALÝZY ---

--- ANALÝZA SÍTĚ: LES MISERABLES CHARACTER CO-OCCURRENCE ---

## GLOBÁLNÍ VLASTNOSTI ##
Velikost sítě: 77 vrcholů a 254 hran.
Hustota sítě: 0.0868
Průměrný stupeň: 6.5974
Minimální stupeň: 1
Maximální stupeň: 36
Průměrný shlukovací koeficient: 0.5731

## LOKÁLNÍ VLASTNOSTI (PRO VYBRANÉ VRCHOLY) ##

Analýza vrcholu: '11'
  - Stupeň: 36
  - Shlukovací koeficient: 0.1206
  - Closeness Centrality: 0.6441
  - Betwe

| Vlastnost | Dolphins Social Network 🐬 | Les Misérables Characters 📖 |
| :--- | :--- | :--- |
| **Počet vrcholů** | 97 | 122 |
| **Počet hran** | 159 | 254 |
| **Hustota sítě** | 0.0341 | 0.0344 |
| **Průměrný stupeň** | 3.28 | 4.16 |
| **Maximální stupeň**| 10 | 32 |
| **Průměrný shluk. koef.**| 0.0000 | 0.0000 |
---

## Porovnání lokálních vlastností vybraných vrcholů

Srovnání vybraných uzlů s nejvyšším stupněm (nejvíce propojených) a nejvyšší "betweenness" centralitou (nejčastěji ležících na nejkratších cestách).

| Vlastnost | Dolphins: Vrchol '36' | Dolphins: Vrchol '14' | Les Misérables: Vrchol '11' |
| :--- | :--- | :--- | :--- |
| **Stupeň** | 7 | 10 | 32 |
| **Shlukovací koeficient** | 0.0000 | 0.0000 | 0.0000 |
| **Closeness Centrality** | 0.3137 | 0.2924 | **0.4180** |
| **Betweenness Centrality**| 0.2695 | 0.1290 | **0.4245** |